<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_GetWorkPath.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# GetWorkPath App  📒
***Create your own Tapis App***
THIS WORKS, KIND OF, BUT IT'S SO CONVOLUTED AND STILL NOT CLEAR.
by Silvia Mazzoni, DesignSafe, 2025

Let's write an app to get your user and system-dependent path

## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
import json

In [2]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


---
## Connect to Tapis

In [3]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-21T03:15:52+00:00
 Token expires in: 3:38:26.569873
-- LOG IN SUCCESSFUL! --


## Define actions!

In [4]:
# Make these True once you are done validating
do_makeApp = True
do_submitJob = True

---
## Configure App

In [5]:
app_id = 'get-work-path-test'
app_description = 'get work path'

### AutoIncrement app_version

In [6]:
if do_makeApp:
    app_version = OpsUtils.increment_tapis_app_version(t,app_id)

app exists, now latest_app_version 0.0.14
now app_version 0.0.15


### Where to store the files

In [7]:
if do_makeApp:
    app_system_id="designsafe.storage.default"
    username = OpsUtils.get_tapis_username(t)
    app_path = f"{username}/apps/{app_id}/{app_version}"
    container_filename = f'{app_id}.zip'

### Define folder structure: app_id/version

In [8]:
if do_makeApp:
    app_folderName = f'~/MyData/myAuthoredTapisApps/{app_id}/{app_version}'; # your choice

In [9]:
if do_makeApp:
    app_folder = os.path.abspath(os.path.expanduser(app_folderName))
    os.makedirs(app_folder, exist_ok=True)
    print(f'app_folder: {app_folder}\n exists:',os.path.exists(app_folder))

app_folder: /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.15
 exists: True


---
## Create the App Files

A Tapis app needs the following **core files**:

* **Readme.MD** – App Description (OPTIONAL)<br>
This file is helpful in communicating content to the app user.  

* **app.json** – App Definition<br>
Defines the app’s metadata, inputs, parameters, and execution configuration.    

* **profile.json** – Environment Setup *(optional but common)*<br>
Loads modules or sets environment variables on the execution system. This file will load modules before your script runs. This is executed on the compute node.

* **tapisjob_app.sh** – Wrapper Script<br>
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [10]:
# let's collect the list of files we are creating as we will need this list in upload
if do_makeApp:
    appFilesList = [];

#### **Readme.MD** – App Description
This file is helpful in communicating content to the app user.  

In [11]:
if do_makeApp:
    thisFilename = 'ReadMe.MD'
    thisText = """\
    # making a dummy app to get user-and-system-specific work path
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **app.json** – App Definition
Defines the app’s metadata, inputs, parameters, and execution configuration.    

In [12]:
if do_makeApp:
    thisFilename = 'app.json'
    thisText = """\
    {
        "id": "__app_id__",
        "name": "__app_id__",
        "version": "__app_version__",
        "description": "__app_description__",
        "owner": "${apiUserId}",
        "jobAttributes": {
            "execSystemConstraints": null,
            "execSystemId": "stampede3",
            "execSystemExecDir": "${JobWorkingDir}",
            "execSystemInputDir": "${JobWorkingDir}",
            "execSystemOutputDir": "${JobWorkingDir}",
            "execSystemLogicalQueue": "skx-dev",
            "archiveSystemId": "stampede3",
            "archiveSystemDir": "HOST_EVAL($WORK)"
            },
        "executionSystem": "designsafe.community.execution",
        "deploymentSystem": "designsafe.storage.default",
        "deploymentPath": "apps/bash-echo/0.0.3",
        "templatePath": "wrapper.sh",
        "executionType": "HPC",
        "runtime": "SINGULARITY",
        "containerImage": "docker://rockylinux:9",
        "jobType": "BATCH",
        "parallelism": "SERIAL",
        "defaultNodes": 1,
        "defaultProcessors": 1,
        "defaultMaxRunTime": "00:05:00",
        "defaultMemory": "1GB",
        "archive": false,
        "archiveOnAppError": true,
        "inputs": [],
        "parameters": []
    }

    """
    thisText = thisText.replace("__app_id__", app_id)
    thisText = thisText.replace("__app_version__", app_version)
    thisText = thisText.replace("__app_description__", app_description)
  
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **profile.json** – Environment Setup
This file will load modules before your script runs. This is executed on the compute node.

In [13]:
if do_makeApp:
    thisFilename = 'profile.json'
    thisText = """\
    {
      "modules": []
    }
    """ 
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **tapisjob_app.sh** – Wrapper Script
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [14]:
if do_makeApp:
    import textwrap, time
    from zipfile import ZipFile, ZIP_DEFLATED, ZipInfo
    
    thisFilename_sh = "tapisjob_app.sh"
    thisFilename = container_filename  # existing var


    bash_script = textwrap.dedent("""""")


    zip_path = os.path.join(app_folder, thisFilename)
    print('zip_path',zip_path)

    # write with executable permission inside the ZIP
    zi = ZipInfo(thisFilename_sh)
    zi.date_time = time.localtime(time.time())[:6]
    zi.compress_type = ZIP_DEFLATED
    zi.external_attr = 0o100755 << 16   # -rwxr-xr-x on a regular file
    
    with ZipFile(zip_path, "w", ZIP_DEFLATED) as z:
        z.writestr(zi, bash_script)
        
    appFilesList.append(thisFilename)

zip_path /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.15/get-work-path-test.zip


#### File Check
Look at the files we have written and check for typos or formatting errors.

In [15]:
if do_makeApp:
    print(appFilesList)
    OpsUtils.show_text_file_in_accordion(app_folder, appFilesList)

['ReadMe.MD', 'app.json', 'profile.json', 'get-work-path-test.zip']


---
## Validate App Locally

In [16]:
if do_makeApp:
    OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['validate_app_folder.py'])

In [17]:
if do_makeApp:
    validation = OpsUtils.validate_app_folder(app_folder,appFilesList)
    if not validation:
        print('Validation Failed: stopping here!!!!')
        a = 3/0

🔍 Validating app folder: /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.15

✅ All required files are present.

📄 App ID: get-work-path-test
📄 App Name: get-work-path-test
📄 Version: 0.0.15
🔧 Parameters: []
📦 Inputs: []
📤 Outputs: []

App Keys: ['id', 'name', 'version', 'description', 'owner', 'jobAttributes', 'executionSystem', 'deploymentSystem', 'deploymentPath', 'templatePath', 'executionType', 'runtime', 'containerImage', 'jobType', 'parallelism', 'defaultNodes', 'defaultProcessors', 'defaultMaxRunTime', 'defaultMemory', 'archive', 'archiveOnAppError', 'inputs', 'parameters', 'modules']

✅ Basic validation complete. App folder looks good!


---
## Deploy the App

### Make the directory inside your 'MyData' on designsafe.storage.default.
The apps in this folder are the ones that area actually uploaded.

In [18]:
if do_makeApp:
    t.files.mkdir(systemId=app_system_id, path=app_path)
    print('app_path',app_path)

app_path silvia/apps/get-work-path-test/0.0.15


### Upload files to your deployment system (e.g., DesignSafe default storage)
Using Tapipy (Python SDK) in a Jupyter Notebook

In [19]:
if do_makeApp:
    for fname in appFilesList:
        fpath = f'{app_folder}/{fname}'
        t.upload(source_file_path=fpath,
                 system_id=app_system_id,
                 dest_file_path=f'{app_path}/{fname}')

#### Verify upload

In [20]:
if do_makeApp:
    print('app_system_id:',app_system_id)
    print('app_path:',app_path)
    appfiles = t.files.listFiles(systemId=app_system_id, path=app_path)
    for thisF in appfiles:
        print(thisF)
        print('')

app_system_id: designsafe.storage.default
app_path: silvia/apps/get-work-path-test/0.0.15

group: 819066
lastModified: 2025-08-20T23:37:27Z
mimeType: application/json
name: app.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.15/app.json
size: 1250
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-work-path-test/0.0.15/app.json


group: 819066
lastModified: 2025-08-20T23:37:28Z
mimeType: application/zip
name: get-work-path-test.zip
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.15/get-work-path-test.zip
size: 130
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-work-path-test/0.0.15/get-work-path-test.zip


group: 819066
lastModified: 2025-08-20T23:37:27Z
mimeType: application/json
name: profile.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.15/profile.json
size: 36
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-wo

---
## Register the App
This creates the actual App record that Jobs can run.

Do this Using Tapipy (Python)

In [21]:
if do_makeApp:
    # Create (or create a new version) of the app
    with open(f'{app_folder}/app.json') as f:
        app_def = json.load(f)
    t.apps.createAppVersion(**app_def)

### Check that app is up

#### List the new app

In [22]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
print(appMetaData)


containerImage: docker://rockylinux:9
created: 2025-08-20T23:37:28.678418Z
deleted: False
description: get work path
enabled: True
id: get-work-path-test
isPublic: False
jobAttributes: 
archiveOnAppError: False
archiveSystemDir: HOST_EVAL($WORK)
archiveSystemId: stampede3
cmdPrefix: None
coresPerNode: 1
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: ${JobWorkingDir}
execSystemId: stampede3
execSystemInputDir: ${JobWorkingDir}
execSystemLogicalQueue: skx-dev
execSystemOutputDir: ${JobWorkingDir}
fileInputArrays: []
fileInputs: []
isMpi: False
maxMinutes: 10
memoryMB: 100
mpiCmd: None
nodeCount: 1
parameterSet: 
appArgs: []
archiveFilter: 
excludes: []
includeLaunchFiles: True
includes: []
containerArgs: []
envVariables: []
logConfig: 
stderrFilename: 
stdoutFilename: 
schedulerOptions: []
subscriptions: []
tags: []
jobType: BATCH
locked: False
maxJobs: 2147483647
maxJobsPerUs

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

In [23]:
job_def = {
    "name": "print-work",
    "appId": app_id,
    "appVersion":app_version,
    "archiveSystemId": "stampede3",
}

In [24]:
resp = t.jobs.submitJob(**job_def)
work_dir = resp.archiveSystemDir
print('work_dir',work_dir)

work_dir /work2/05072/silvia/stampede3


In [25]:
print(resp)


_fileInputsSpec: None
_parameterSetModel: None
appId: get-work-path-test
appVersion: 0.0.15
archiveCorrelationId: None
archiveOnAppError: False
archiveSystemDir: /work2/05072/silvia/stampede3
archiveSystemId: stampede3
archiveTransactionId: None
blockedCount: 0
cmdPrefix: None
condition: None
coresPerNode: 1
created: 2025-08-20T23:37:28.847286832Z
createdby: silvia
createdbyTenant: designsafe
description: get-work-path-test-0.0.15 submitted by silvia@designsafe
dtnInputCorrelationId: None
dtnInputTransactionId: None
dtnOutputCorrelationId: None
dtnOutputTransactionId: None
dtnSystemId: None
dtnSystemInputDir: None
dtnSystemOutputDir: None
dynamicExecSystem: False
ended: None
execSystemConstraints: None
execSystemExecDir: /scratch/05072/silvia/tapis/bd769369-7cb0-418e-aa48-18de4b8ded88-007
execSystemId: stampede3
execSystemInputDir: /scratch/05072/silvia/tapis/bd769369-7cb0-418e-aa48-18de4b8ded88-007
execSystemLogicalQueue: skx-dev
execSystemOutputDir: /scratch/05072/silvia/tapis/bd769

In [26]:
# OpsUtils.run_tapis_job(t,tapisInput,job_description = job_def)